# **Amazon Reviews Dataset**

In [0]:
from pprint import pprint

import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd
# import delta_sharing


In [0]:
# # get data
# profile_file = "config.share"
# client = delta_sharing.SharingClient(profile_file)
# print("Available tables:")
# pprint(client.list_all_tables())

# table_url = profile_file + "#share__products.silver.amazon_reviews_silver"
# # limit number of reviews to 10k
# data = delta_sharing.load_as_pandas(table_url, limit=10_000)
# # cast date column to datetime type

df = spark.table("products.silver.amazon_reviews_silver").limit(10_000).toPandas()
data["date"] = pd.to_datetime(data["date"])

In [0]:
print("Dataset shape", data.shape)

data.head()

In [0]:
# print some reviews
def print_reviews(df, n=3):
    for idx, row in data.sample(n).iterrows():
        print("Review Summary:", row["summary"], "\n")
        print(row["reviewText"])
        print("=" * 50)


print_reviews(data)

**Insights**:
    
- **Build a model that identifies whether a review contains text that is useful**
    - ¿What is a good definition for a useful review?

---

In [0]:
# unique values of asin column
asin_vals = data["asin"].value_counts()
asin_vals

In [0]:
asin_vals.describe().astype(int)

**Insights**
- Products range from having 1 to a few thousand reviews.
- Most of the products have 8 or less reviews.

In [0]:
fixed_product = data[data["asin"] == asin_vals.index[0]].head()
fixed_product.head()

In [0]:
print_reviews(fixed_product)

---

In [0]:
duplicated = data[data.duplicated(subset=["asin", "reviewText"], keep=False)].sort_values("reviewText")
duplicated

In [0]:
num_dup = data.duplicated(subset=["asin", "reviewText"], keep="first").sum()
print(f"Percentaje of duplicated reviews {num_dup / len(data) * 100:.2f}%")

**Insights**

- There is definitively reviews that are exactly the same.
- ~1% of the reviews are duplicated.

---

In [0]:
# filter empty reviews
reviews_len = data["reviewText"][~data["reviewText"].isnull()].map(len)

# reviews len with outliers
plt.figure(figsize=(6, 4))
plt.boxplot(reviews_len, vert=False)
plt.title("Reviews length with Outliers")
plt.xlabel("column_name")
plt.show()

Q1 = reviews_len.quantile(0.25)
Q3 = reviews_len.quantile(0.75)
IQR = Q3 - Q1

reviews_no_outliers = reviews_len[(reviews_len >= Q1 - 1.5 * IQR) & (reviews_len <= Q3 + 1.5 * IQR)]

# plot reviews without outliers
plt.figure(figsize=(6, 4))
plt.boxplot(reviews_no_outliers, vert=False)
plt.title("Reviews length without Outliers")
plt.xlabel("column_name")
plt.show()

**Insights**
- There are some reviews with extremely long texts.
- Most reviews have between a couple tens to a couple thousands of characters

---

In [0]:
data["date"].describe()

---

In [0]:
# mising values
msno.matrix(data);

In [0]:
print("Percentaje of missing values:")

data.isnull().sum() / len(data) * 100

**Insights**
- There are reviews with no text. Though we couuld still use the rating information.
- Most reviews don't have images.
- `Vote` column has a lot of missing values.

---

In [0]:
conteo_puntuaciones = data["overall"].value_counts()

plt.bar(conteo_puntuaciones.index, conteo_puntuaciones.values)
plt.xlabel("Puntuación")
plt.ylabel("Cantidad")
plt.title("Distribución de puntuaciones")
plt.xticks(conteo_puntuaciones.index)
plt.show()

In [0]:
data["verified"].value_counts().plot(kind="bar", title="Verified vs Not Verified reviews");

In [0]:
conteo_puntuaciones = data_vtrue["overall"].value_counts()

plt.bar(conteo_puntuaciones.index, conteo_puntuaciones.values)
plt.xlabel("Puntuación")
plt.ylabel("Cantidad")
plt.title("Distribución de puntuaciones con verificacion")
plt.xticks(conteo_puntuaciones.index)
plt.show()

---

In [0]:
plt.figure(figsize=(20, 4))
data["date"].value_counts().plot(title="Number of reviews");

In [0]:
plt.figure(figsize=(20, 4))
data["date"][data["date"].dt.year > 2012].value_counts().plot(title="Number of reviews");

In [0]:
data["date"].dt.weekday.value_counts().sort_index().plot(kind="bar", title="Day of the week");

In [0]:
data["date"].dt.day.value_counts().sort_index().plot(kind="bar", title="Day of the month");

In [0]:
data["date"].dt.year.value_counts().sort_index().plot(kind="bar", title="Year");